In [4]:
import pandas as pd



In [5]:
import numpy as np
import matplotlib.pyplot  as plt

In [11]:
df=pd.read_csv('../data/highly_relevant_posts_descending_threshold_50.csv')

In [7]:
from drug_named_entity_recognition import find_drugs

[({'name': 'Acetaminophen', 'synonyms': {'Panadol', 'Acamol', 'Hydroxyacetanilide', 'Acephen', 'Datril', 'Ofirmev', 'Tylenol', 'Acetaco', 'Actamin', 'Algotropyl', 'Paracetamol', 'Acetominophen', 'Paracetamolum', 'Acetaminophen', 'Acetamidophenol', 'Acenol'}, 'medline_plus_id': 'a621016', 'generic_names': ['Acetaminophen'], 'mesh_id': 'D058633', 'drugbank_id': 'DB00316', 'wikipedia_url': 'https://en.wikipedia.org/wiki/Paracetamol'}, 2, 2)]


In [8]:
def get_drug_list(text):
    drug_tuples = find_drugs(text.split(" "), is_ignore_case=True)
    drug_names = [drug[0] for drug in drug_tuples]
    drug_list= [drug.get('name') for drug in drug_names]
    unique_drug_list = sorted(set(drug_list))
    return unique_drug_list

In [12]:
df['drug_list'] =df['processed_text'].apply(get_drug_list)

In [13]:
from collections import defaultdict


def build_medication_index(df, column_name):
    med_index = defaultdict(list)
    for idx, row in df.iterrows():
        for med in row[column_name]:
            med_index[med].append(idx)
    return med_index

def find_medication_with_index(med_index, df, medication_name):
    if medication_name in med_index:
        matching_indices = med_index[medication_name]
        return df.loc[matching_indices]
    else:
        return pd.DataFrame()  # Return empty DataFrame if medication not found

# Build the index
med_index = build_medication_index(df, 'drug_list')

# Call the function with a specific medication name
medication_name = 'Lamotrigine'
result = find_medication_with_index(med_index, df, medication_name)
print(result)


                                                  title  \
0     Informal survey: What (legal) drugs, if any, '...   
3               My experience with lamictal/lamotrigine   
8     What medication(s) work best for when you have...   
12    My first time Inpatient: how I learned to advo...   
14    Tomorrow I am seeing my doctor pertaining to m...   
...                                                 ...   
2494                 Lamictal experiences/side effects?   
2495                                    Lamictal Online   
2544                          Paranoia is getting worse   
2545           TW - have i took enough to cause damage?   
2552  Do you ever feel like your medication is actua...   

                                               selftext  created_utc  over_18  \
0     hi! so i'm officially diagnosed, tried dbt but...   1602953594    False   
3     this is going to be a very long post because i...   1628777635    False   
8     i am formally diagnosed with severe genera

In [14]:
def get_synonyms(medication):
    drug_tuples = find_drugs(medication.split(" "), is_ignore_case=True)
    drug_names = [drug[0] for drug in drug_tuples]
    drug_list= [drug.get('synonyms') for drug in drug_names]
    return drug_list

In [15]:
def synonym_list_string(medication):
    return str(get_synonyms(medication_name)).strip('[,],{,}')

In [17]:
import openai

# Function to get sentiment and reason for a given medication from text
def get_medication_sentiment(medication_name, text):
    prompt = f"This passage enclosed in `` `` contains some discussion about the medication {medication_name} (perhaps referring to them in terms of a synonym from the list " + synonym_list_string(medication_name)  + "). The response should only consider this medication and none others. Please format the output as a dictionary with the following keys, 'sentiment', 'reason'. For 'sentiment,' respond with '1' if the medication helped to treat the poster, even if there are minor side effects or the medication did not fully resolve the condition. Respond with '-1' if the medication did not help the poster. Respond with '0' if the discussion of the medication is neutral and respond with '-' if there is not enough information to determine this information. 'Reason' should be a short sentence about the experience the poster had with the medication, including effectiveness, side effects, interactions with other medications, or addictions:``{text}``"
    response = openai.Completion.create(
        engine="gpt-3.5-turbo-0125",
        prompt=prompt,
        max_tokens=150,
        temperature=0.5,
        stop=None
    )
    output = response.choices[0].text.strip()
    sentiment, reason = output.split('\n')
    return {'sentiment': sentiment, 'reason': reason}


# Function to process DataFrame
def process_dataframe(df, medication_name):
    rows_with_medication = df[df['drug_names'].apply(lambda x: medication_name in x)]
    sentiments = []
    reasons = []
    for index, row in rows_with_medication.iterrows():
        text = row['combined_text']
        sentiment_reason = get_medication_sentiment(medication_name, text)
        sentiments.append(sentiment_reason['sentiment'])
        reasons.append(sentiment_reason['reason'])
    rows_with_medication['sentiment'] = sentiments
    rows_with_medication['reason'] = reasons
    return rows_with_medication
